# Menganalisis Risiko Gagal Bayar Peminjam

Tugas kita adalah menyiapkan laporan untuk divisi kredit suatu bank. Kita akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan kita akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

Tujuan: Mencari tahu apakah status perkawinan dan jumlah anak memengaruhi probabilitas dalam pelunasan pinjaman.

Hipotesis: 
- Nasabah yang gagal bayar pinjaman didominasi oleh nasabah dengan status menikah.
- Nasabah yang memiliki anak rata-rata mengalami gagal bayar pinjaman.

## Membuka file data dan membaca informasi umum.

In [409]:
# Muat semua library
import pandas as pd

In [410]:
# Muat datanya
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Eksplorasi data

**Deskripsi data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - status perkawinan
- `family_status_id` - pengidentifikasi untuk status perkawinan nasabah
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman


In [411]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
df.shape

(21525, 12)

In [412]:
# Mari tampilkan 5 baris pertama
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Dari data dia ats terdapat kolom 'days_employed' yang memiliku nilai negative. Hal ini patut diselidiki mengingat pengalaman kerja tidak mungkin bernilai negative.

In [413]:
# Dapatkan informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ada nilai yang hilang di kolom "days_employed" dan "total_income". Jumlah yang hilang juga simetris.

In [414]:
# Mari kita lihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Nilai yang hilang tampak simetris, kita akan berasumsi bahwa jika ada nilai hilang di kolom "days_employed" maka ada nilai yang hilang juga di kolom "totak_income", kemungkinan dikarenakan nasabah masih bersekolah.

In [415]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.

df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Kesimpulan sementara**

Jumlah baris sama dengan jumlah nilai yang hilang, sehingga dipastikan bahwa jika ada kolom "days_employed" yang hilang maka dipastikan kolom "total_income" juga hilang.

Nilai yang hilang sebesar 10% sehingga bisa dibilang lumayan besar, sehingga lebih baik kita isi nilai yang hilang. Penyebab nilai hilang bisa saja karena jenis pekerjaan yang tidak tetap (part time) atau pekerjaan yang dilaksanakan saat ada project saja, sehingga relate ke income juga tidak tetap.


In [416]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
df.loc[df['children'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [417]:
# Periksalah distribusinya

percentage = (df['total_income'].isnull().sum() / len(df))
print("percentage of missing value is {:.0%}".format(percentage))


percentage of missing value is 10%



**Kemungkinan penyebab hilangnya nilai dalam data**

Hilangnya value terdapat pola, yaitu kalau nilai days_employed hilang, kolom total_income juga ikut hilang.


In [418]:
# Memeriksa distribusi di seluruh dataset

df.isna().sum()/len(df)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Kesimpulan sementara**

Jumlah nilai yang hilang dalam data sebesar 10% yangmana agak besar.

In [419]:
df.duplicated().sum()

54

Terdapat duplikat dan hanya satu saja sehingga tidak terlalu berpengaruh terhadap data kita.

## Transformasi data

Mari kita perhatikan setiap kolom untuk melihat masalah apa yang mungkin dimiliki mereka.

In [420]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [421]:
# Perbaiki pencatatan jika diperlukan
df['education']=df['education'].str.lower()

In [422]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
df['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Memeriksa data kolom `children`

In [423]:
# Mari kita lihat distribusi nilai pada kolom `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Terdapat nilai jumlah anak 20, rasanya tidak mungkin, mungkin karena salah ketik.
Jumlah anak negative juga tidak mungkin, mungkin karena salah ketik.

In [424]:
# [perbaiki data berdasarkan keputusanmu]
df['children']=df['children'].abs()
df['children'] = df['children'].replace(20,2)


In [425]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
df['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Memeriksa kolom `days_employed`

In [426]:
# Mencari data yang bermasalah di kolom `days_employed` jika memang terdapat masalah dan menghitung persentasenya
print(f"percentage : {(df['days_employed']<0).sum()/ len(df) *100} %")

percentage : 73.89547038327527 %


In [427]:
# Atasi nilai yang bermasalah, jika ada
df['days_employed'] = round(df['days_employed'].abs())

In [428]:
df['years_employed'] = df['days_employed']/365

In [429]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.117808
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.027397
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.405479
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.301370
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235616


In [432]:
df['days_employed']

0          8438.0
1          4025.0
2          5623.0
3          4125.0
4        340266.0
           ...   
21520      4529.0
21521    343937.0
21522      2113.0
21523      3112.0
21524      1985.0
Name: days_employed, Length: 21525, dtype: float64

Memeriksa kolom `dob_years`

In [325]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
df.loc[df['dob_years']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
99,0,346542.0,0,secondary education,1,married,0,F,retiree,0,11406.644,car,949.430137
149,0,2664.0,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions,7.298630
270,3,1873.0,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation,5.131507
578,0,397857.0,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property,1090.019178
1040,0,1158.0,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car,3.172603
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing,NaN
20462,0,338735.0,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house,928.041096
20577,0,331741.0,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property,908.879452
21179,2,109.0,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate,0.298630


Terdapat  usia yang tertulis 0 sebanyak 101 data

In [326]:
#percentage
len(df.loc[df['dob_years']==0])/ len(df) *100

0.4692218350754936

Kita akan mengisi kolom `dob_years` dengan rata-rata

In [434]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
average_age = int(df['dob_years'].mean())

In [435]:
df.loc[df['dob_years']==0, 'dob_years']=average_age

In [436]:
df['year_working']= df['dob_years']-18

In [437]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df.loc[df['year_working']<0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working


Memeriksa kolom `family_status`

In [331]:
# Mari kita lihat nilai untuk kolom ini
df['family_status'].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Memeriksa kolom `gender`

In [334]:
# Mari kita liat nilai dalam kolom ini
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [335]:
# Mengatasi nilai-nilai yang bermasalah
df = df[df['gender'] != 'XNA']

In [336]:
# Memeriksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Memeriksa kolom `income_type`

In [337]:
# Mari kita lihat nilai dalam kolom ini
df['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [338]:
# Atasi nilai yang bermasalah, jika ada
def replace_income_type(data, wrong, correct):
    data = data.replace(wrong, correct)
    return data
df['income_type'] = replace_income_type(data=df['income_type'], wrong='entrepreneur', correct='business')
df['income_type'] = replace_income_type(data=df['income_type'], wrong='student', correct='unemployed')
df['income_type'] = replace_income_type(data=df['income_type'], wrong='paternity / maternity leave', correct='unemployed')


In [339]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['income_type'].value_counts()


employee         11119
business          5086
retiree           3856
civil servant     1459
unemployed           4
Name: income_type, dtype: int64

In [340]:
#Permasalahan di kolom PURPOSE
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [341]:
# Atasi nilai yang bermasalah, jika ada
housing = ['construction of own property','real estate transactions','transactions with my real estate',
           'building a real estate','transactions with commercial real estate','buy residential real estate',
           'buy commercial real estate','buy real estate','purchase of the house','housing transactions',
           'purchase of the house for my family','housing','purchase of my own house','housing renovation',
          'buying property for renting out','building a property','property']
car = ['car purchase','buying a second-hand car','buying my own car','transactions with my real estate', 
       'cars','second-hand car purchase','car','to own a car','purchase of a car','to buy a car']
edu = ['supplementary education','education','to become educated','getting an education','to get a supplementary education',
       'getting higher education','profile education','university education','going to university']
wed = ['to have a wedding','having a wedding','wedding ceremony']
def replace_purpose(data, wrong, correct):
    data = data.replace(wrong, correct)
    return data
df['purpose'] = replace_purpose(data=df['purpose'], wrong=housing, correct='housing')
df['purpose'] = replace_purpose(data=df['purpose'], wrong=car, correct='car')
df['purpose'] = replace_purpose(data=df['purpose'], wrong=edu, correct='education')
df['purpose'] = replace_purpose(data=df['purpose'], wrong=wed, correct='wedding')

In [342]:
df['purpose'].unique()

array(['housing', 'car', 'education', 'wedding'], dtype=object)

Mengecek apakah ada duplikat di dalam data kita.

In [343]:
# Periksa duplikat
df.duplicated().sum()

408

In [344]:
# Atasi duplikat, jika ada
df = df.drop_duplicates().reset_index(drop=True)

In [345]:
# Lakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
df.duplicated().sum()

0

In [346]:
# Periksa ukuran dataset yang sekarang kamu miliki setelah manipulasi pertama yang kamu lakukan
df.shape

(21116, 14)

# Bekerja dengan nilai yang hilang

In [347]:
# Menemukan dictionary
marital_status_dict = df[['family_status','family_status_id']]
marital_status_dict = marital_status_dict.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id', ascending=True)
marital_status_dict

,family_status,family_status_id
0,married,0
1,civil partnership,1
2,widow / widower,2
3,divorced,3
4,unmarried,4


In [348]:
edu_dict = df[['education','education_id']]
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True).sort_values(by='education_id', ascending=True)
edu_dict

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


In [349]:
#Kita coba cek status apakah yang paling banyak gagal bayar?
pd.DataFrame(df.groupby('family_status')['debt'].value_counts())

debt
family_status     debt       
civil partnership 0      3734
                  1       388
divorced          0      1108
                  1        85
married           0     11145
                  1       929
unmarried         0      2510
                  1       274
widow / widower   0       880
                  1        63

In [350]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,housing,23.117808,24
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,11.027397,18
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,housing,15.405479,15
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,11.301370,14
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,932.235616,35


### Memperbaiki nilai yang hilang di `total_income`

Membuat kategori usia untuk nasabah. Buat kolom baru yang memuat kategori usia. Strategi ini dapat membantu untuk menghitung total nilai pendapatan.


In [351]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def age_group (age):
    if age <= 25:
        value = '0-25 YO'
    elif age > 25 and age <= 50:
        value = '26-50 YO'
    else:
        value= '>50 YO'
    return value

In [352]:
# Lakukan pengujian untuk melihat apakah fungsimu bekerja atau tidak
age_group(34)

'26-50 YO'

In [353]:
# Buatlah kolom baru berdasarkan fungsi
df['age_group']=df['dob_years'].apply(age_group)


In [354]:
# Periksa bagaimana nilai di dalam kolom baru
df['age_group'].value_counts()

26-50 YO    13395
>50 YO       6499
0-25 YO      1222
Name: age_group, dtype: int64

Membuat tabel yang hanya memuat data tanpa nilai yang hilang. Data ini akan digunakan untuk memperbaiki nilai yang hilang.

In [355]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
notnull_income = df.loc[~df['total_income'].isnull()]

In [356]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah kita identifikasi
#notnull_income.groupby('age_group')['total_income'].mean()
pd.pivot_table(notnull_income, index='age_group', values='total_income', aggfunc='mean').reset_index()

,age_group,total_income
0,0-25 YO,23447.795802
1,26-50 YO,28103.675052
2,>50 YO,24704.150073


In [357]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah kita identifikasi
age_income_group = pd.pivot_table(notnull_income, index='age_group', values='total_income', aggfunc='median').reset_index()
age_income_group

,age_group,total_income
0,0-25 YO,21423.8355
1,26-50 YO,24532.2110
2,>50 YO,21310.9445


In [358]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang
def replace_income(dataset, grouping):
    for group in grouping['age_group']:
        index = 0
        dataset.loc[(dataset['age_group']==group)&(dataset['total_income'].isnull()),'total_income'] = grouping.iloc[index,1]
        index = index + 1
    return dataset

In [359]:
# Memeriksa bagaimana nilai di dalam kolom baru
replace_income(dataset=df, grouping=age_income_group)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,housing,23.117808,24,26-50 YO
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,11.027397,18,26-50 YO
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,housing,15.405479,15,26-50 YO
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,11.301370,14,26-50 YO
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,932.235616,35,>50 YO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21111,1,4529.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing,12.408219,25,26-50 YO
21112,0,343937.0,67,secondary education,1,married,0,F,retiree,0,24959.969,car,942.293151,49,>50 YO
21113,1,2113.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,housing,5.789041,20,26-50 YO
21114,3,3112.0,38,secondary education,1,married,0,M,employee,1,39054.888,car,8.526027,20,26-50 YO


In [360]:
# Terapkan fungsi tersebut ke setiap baris
df = replace_income(dataset=df, grouping=age_income_group)

In [361]:
# Periksa apakah kita mendapatkan kesalahan
df.isnull().sum()

children               0
days_employed       1766
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
years_employed      1766
year_working           0
age_group              0
dtype: int64

In [363]:
# Periksa jumlah entri di kolom
df.count()


children            21116
days_employed       19350
dob_years           21116
education           21116
education_id        21116
family_status       21116
family_status_id    21116
gender              21116
income_type         21116
debt                21116
total_income        21116
purpose             21116
years_employed      19350
year_working        21116
age_group           21116
dtype: int64

###  Memperbaiki nilai di `days_employed`

In [364]:
# Distribusi median dari `days_employed` berdasarkan parameter yang kamu identifikasi
age_median_group = pd.pivot_table(notnull_income, index='age_group', values='days_employed', aggfunc='median').reset_index()
age_median_group

,age_group,days_employed
0,0-25 YO,797.5
1,26-50 YO,1700.0
2,>50 YO,333910.5


In [365]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang kamu identifikasi
age_average_group = pd.pivot_table(notnull_income, index='age_group', values='days_employed', aggfunc='mean').reset_index()
age_average_group

,age_group,days_employed
0,0-25 YO,1223.339029
1,26-50 YO,9409.659276
2,>50 YO,197331.070184


In [370]:
# Ganti nilai yang hilang
def replace_days_employed(dataset, grouping):
    for group in grouping['age_group']:
        index = 0
        dataset.loc[(dataset['age_group']==group)&(dataset['days_employed'].isnull()),'days_employed'] = grouping.iloc[index,1]
        index = index + 1
    return dataset

In [371]:
replace_days_employed(dataset=df, grouping=age_average_group)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,housing,23.117808,24,26-50 YO
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,11.027397,18,26-50 YO
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,housing,15.405479,15,26-50 YO
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,11.301370,14,26-50 YO
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,932.235616,35,>50 YO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21111,1,4529.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing,12.408219,25,26-50 YO
21112,0,343937.0,67,secondary education,1,married,0,F,retiree,0,24959.969,car,942.293151,49,>50 YO
21113,1,2113.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,housing,5.789041,20,26-50 YO
21114,3,3112.0,38,secondary education,1,married,0,M,employee,1,39054.888,car,8.526027,20,26-50 YO


Memeriksa apakah jumlah total nilai di kolom ini sesuai dengan jumlah nilai di kolom lain.

In [372]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
df.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
years_employed      1766
year_working           0
age_group              0
dtype: int64

In [373]:
df['days_employed'] = round(df['days_employed'].abs())
df['years_employed'] = df['days_employed']/365
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
year_working        0
age_group           0
dtype: int64

In [374]:
#Memperbaiki nilai yang tidak wajar di kolom "days_employed", 
#setelah diconvert ke tahun terdapat nilai tidak wajar seperti 900 tahun.

df.loc[df['days_employed']>30000]  #Pakai >30000 hari karena tidak mungkin seseorang telah bekerja lebih dari 80 tahun.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,932.235616,35,>50 YO
18,0,400281.0,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,car,1096.660274,35,>50 YO
24,1,338552.0,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,housing,927.539726,39,>50 YO
25,0,363548.0,67,secondary education,1,married,0,M,retiree,0,8818.041,housing,996.021918,49,>50 YO
30,1,335582.0,62,secondary education,1,married,0,F,retiree,0,27432.971,housing,919.402740,44,>50 YO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21096,0,338905.0,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,wedding,928.506849,35,>50 YO
21099,0,386498.0,62,secondary education,1,married,0,M,retiree,0,11622.175,housing,1058.898630,44,>50 YO
21100,0,362161.0,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,housing,992.221918,41,>50 YO
21109,0,373996.0,59,secondary education,1,married,0,F,retiree,0,24618.344,car,1024.646575,41,>50 YO


In [375]:
age_median_group = pd.pivot_table(notnull_income, index='age_group', values='days_employed', aggfunc='median').reset_index()
age_median_group

,age_group,days_employed
0,0-25 YO,797.5
1,26-50 YO,1700.0
2,>50 YO,333910.5


In [376]:
#Mengganti nilai yang tidak wajar di kolom "years_employed" dengan median.
def replace_uncommon_days_employed(dataset, grouping):
    for group in grouping['age_group']:
        index = 0
        dataset.loc[(dataset['age_group']==group)&(dataset['days_employed']>=30000),'days_employed'] = grouping.iloc[index,1]
        index = index + 1
    return dataset

In [377]:
replace_uncommon_days_employed(dataset=df, grouping=age_median_group)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,housing,23.117808,24,26-50 YO
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,11.027397,18,26-50 YO
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,housing,15.405479,15,26-50 YO
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,11.301370,14,26-50 YO
4,0,797.5,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,932.235616,35,>50 YO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21111,1,4529.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing,12.408219,25,26-50 YO
21112,0,797.5,67,secondary education,1,married,0,F,retiree,0,24959.969,car,942.293151,49,>50 YO
21113,1,2113.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,housing,5.789041,20,26-50 YO
21114,3,3112.0,38,secondary education,1,married,0,M,employee,1,39054.888,car,8.526027,20,26-50 YO


In [378]:
df['years_employed'] = df['days_employed']/365

In [379]:
#Memastikan kembali kolom "years_employed" tidak ada nilai yang aneh seperti 900 tahun dan lainnya.
replace_uncommon_days_employed(dataset=df, grouping=age_median_group)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,housing,23.117808,24,26-50 YO
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,11.027397,18,26-50 YO
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,housing,15.405479,15,26-50 YO
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,11.301370,14,26-50 YO
4,0,797.5,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,2.184932,35,>50 YO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21111,1,4529.0,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing,12.408219,25,26-50 YO
21112,0,797.5,67,secondary education,1,married,0,F,retiree,0,24959.969,car,2.184932,49,>50 YO
21113,1,2113.0,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,housing,5.789041,20,26-50 YO
21114,3,3112.0,38,secondary education,1,married,0,M,employee,1,39054.888,car,8.526027,20,26-50 YO


## Pengkategorian data


In [380]:
# Tampilkan nilai data yang kamu pilih untuk pengkategorian
df['total_income']=round(df['total_income'])

Mari kita periksa nilai unik

In [381]:
# Periksa nilai unik
df['total_income'].unique()

array([40620., 17933., 23342., ..., 35967., 24960., 14348.])

In [382]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def income_group(total_income):
    if total_income <=20000:
        value = 'category A (<=20000)'
    elif total_income > 20000 and total_income <=40000:
        value = 'category B(20000 to 40000)'
    else:
        value = 'category C(abv 40000)'
    return value

In [383]:
# Buat kolom yang memuat kategori dan hitung nilainya
df['income_group'] = df['total_income'].apply(income_group)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,year_working,age_group,income_group
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.0,housing,23.117808,24,26-50 YO,category C(abv 40000)
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17933.0,car,11.027397,18,26-50 YO,category A (<=20000)
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23342.0,housing,15.405479,15,26-50 YO,category B(20000 to 40000)
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42821.0,education,11.301370,14,26-50 YO,category C(abv 40000)
4,0,797.5,53,secondary education,1,civil partnership,1,F,retiree,0,25379.0,wedding,2.184932,35,>50 YO,category B(20000 to 40000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21111,1,4529.0,43,secondary education,1,civil partnership,1,F,business,0,35967.0,housing,12.408219,25,26-50 YO,category B(20000 to 40000)
21112,0,797.5,67,secondary education,1,married,0,F,retiree,0,24960.0,car,2.184932,49,>50 YO,category B(20000 to 40000)
21113,1,2113.0,38,secondary education,1,civil partnership,1,M,employee,1,14348.0,housing,5.789041,20,26-50 YO,category A (<=20000)
21114,3,3112.0,38,secondary education,1,married,0,M,employee,1,39055.0,car,8.526027,20,26-50 YO,category B(20000 to 40000)


In [384]:
df['income_group'].value_counts()

category B(20000 to 40000)    10934
category A (<=20000)           7370
category C(abv 40000)          2812
Name: income_group, dtype: int64

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [390]:
# Periksa data anak dan data gagal bayar pinjaman
df['children'].value_counts()

0    13826
1     4797
2     2115
3      329
4       40
5        9
Name: children, dtype: int64

In [391]:
# Periksa data anak dan data gagal bayar pinjaman
df['debt'].value_counts()

0    19377
1     1739
Name: debt, dtype: int64

In [392]:
data_debt_1 = df.loc[df['debt']==1]

In [393]:
data_debt_1['children'].value_counts()

0    1061
1     445
2     202
3      27
4       4
Name: children, dtype: int64

In [394]:
# Hitung persentase gagal bayar berdasarkan jumlah anak
print(f"persentase gagal bayar dan punya 0 anak adalah {1061/len(data_debt_1)*100}%")

persentase gagal bayar dan punya 0 anak adalah 61.01207590569293%


**Kesimpulan**

Berdasarkan data di atas, nasabah yang gagal mayoritas yang tidak memiliki anak.


**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [395]:
# Periksa data status keluarga dan data gagal bayar pinjaman

df['family_status'].value_counts()

married              12074
civil partnership     4122
unmarried             2784
divorced              1193
widow / widower        943
Name: family_status, dtype: int64

In [396]:
data_debt_1['family_status'].value_counts()

married              929
civil partnership    388
unmarried            274
divorced              85
widow / widower       63
Name: family_status, dtype: int64

In [397]:
# Hitung persentase gagal bayar berdasarkan status keluarga
print(f"persentase gagal bayar dengan status 'married' adalah {929/len(data_debt_1)*100}%")

persentase gagal bayar dengan status 'married' adalah 53.42150661299597%


**Kesimpulan**

Dari data di atas, nasabah yang gagal bayar statusnya sudah menikah.


**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [398]:
# Periksa data tingkat pendapatan dan data gagal bayar pinjaman

data_debt_1['income_group'].value_counts()


category B(20000 to 40000)    937
category A (<=20000)          608
category C(abv 40000)         194
Name: income_group, dtype: int64

In [399]:
# Hitung persentase gagal bayar berdasarkan tingkat pendapatan
print(f"persentase gagal bayar ada pada category B yaitu {937/len(data_debt_1)*100}%")

persentase gagal bayar ada pada category B yaitu 53.881541115583666%


**Kesimpulan**

[Tulis kesimpulanmu berdasarkan manipulasi dan pengamatanmu.]
Nasabah yang gagal bayar berada di rentang penghasilan 20000 - 40000

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [400]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
data_debt_1['purpose'].value_counts()


housing      781
car          402
education    370
wedding      186
Name: purpose, dtype: int64

In [401]:
print(f"persentase gagal bayar untuk tujuan housing yaitu {781/len(data_debt_1)*100}%")

persentase gagal bayar untuk tujuan housing yaitu 44.910868315123636%


**Kesimpulan**

Dari data di atas, nasabah yang gagal bayar dengan tujuan housing menjadi penyebab utama.



# Kesimpulan umum 

Berdasarkan pra-pemrosesan data yang dilakukan terdapat beberapa keanehan:
1. Nilai dari "days_employed" berupa negative, sehingga perlu diubah menjadi positive.
2. Terdapat umur nasabah 0, sehingga perlu diganti dengan usia rata-rata.
3. Terdapat jumlah anak 20 dan -1, diperkirakan karena kesalahan ketik sehingga perlu diperbaiki.
4. Terdapat nilai yang hilang di kolom "days_employed" dan "total_income" secara simetris, sehingga perlu diisi dengan rata-rata sesuai kategori. Hal ini kemungkinan nasabah masih bersekolah, atau bekerja part-time.

KESIMPULAN:
1. Jumlah anak tidak terlalu memengaruhi probabilitas gagal bayar pinjaman, karena dari data yang didapatkan justru paling banyak gagal bayar adalah nasabah yang tidak punya anak.
2. Mayoritas nasabah yang gagal bayar statusnya menikah.
3. Nasabah yang gagal bayar rata-rata memiliki penghasilan 20000-40000
4. Tujuan pinjaman berupa kategori "housing" menjadi penyebab tertinggi nasabah gagal bayar.
